In [1]:
"""
This is a skeleton of Assignment 1 for DS5110/CS5501 Spring 2024 (UVA).
NOTE you will need to change the .ipynb file name by following the naming convention.
Code should be commented well. 
Feel free to import any missing packages or add extra cells as you see fit. 
"""
import dask
from dask.distributed import Client
import dask.dataframe as dd
import time
import json
import numpy as np
import pandas as pd

In [2]:
# TODO: fill out the private IP address of your Dask scheduler and port here when creating a Dask client object
client = Client('172.31.89.180:8786')
client = client.restart()
print(client)

None


In [3]:
%%time

# Load the .CSV files into the Dask dataframes
questions = dd.read_csv('/home/ubuntu/questions.csv', sample=2**30, assume_missing=True)
question_tags = dd.read_csv('/home/ubuntu/question_tags.csv', sample=2**30, assume_missing=True)

CPU times: user 558 ms, sys: 2.44 s, total: 3 s
Wall time: 2.98 s


In [4]:
%%time
# Task 1

# Function to compute missing value percentages in Dask
def missing_percentage(df):
    return (df.isnull().sum() / df.shape[0]).compute() * 100

# Compute missing values for both tables
missing_questions = missing_percentage(questions)
missing_question_tags = missing_percentage(question_tags)

# Display results
print("Missing values in questions.csv:")
print(missing_questions)

print("\nMissing values in question_tags.csv:")
print(missing_question_tags)

Missing values in questions.csv:
Id               0.000000
CreationDate     0.000000
ClosedDate      89.973578
DeletionDate    76.962907
Score            0.000000
OwnerUserId     23.920885
AnswerCount     10.114036
dtype: float64

Missing values in question_tags.csv:
Id     0.000000
Tag    0.027501
dtype: float64
CPU times: user 83.9 ms, sys: 15.9 ms, total: 99.7 ms
Wall time: 30.2 s


In [5]:
%%time
# Task 2

# Compute statistics for the 'Score' column using Dask
score_stats = {
    "Mean": questions["Score"].mean().compute(),
    "Standard Deviation": questions["Score"].std().compute(),
    "Median": questions["Score"].quantile(0.5).compute(),  # Use quantile for median in Dask
    "Min": questions["Score"].min().compute(),
    "Max": questions["Score"].max().compute()
}

# Display results
for stat, value in score_stats.items():
    print(f"{stat}: {value}")

Mean: 1.2041671665555287
Standard Deviation: 15.64573552737141
Median: 1.0
Min: -154.0
Max: 16902.0
CPU times: user 145 ms, sys: 17 ms, total: 162 ms
Wall time: 1min 2s


In [6]:
%%time
# Task 3

# Count occurrences of each tag and compute the top 5
top_tags = question_tags["Tag"].value_counts().compute().head(5)

# Display results
print("Top 5 tags with the highest number of questions:")
print(top_tags)

Top 5 tags with the highest number of questions:
Tag
#                       1
.net-interview          5
.net-standard-1.5       2
10gen-csharp-driver    12
2to3                    8
Name: count, dtype: int64[pyarrow]
CPU times: user 51.9 ms, sys: 6.07 ms, total: 58 ms
Wall time: 15.5 s


In [7]:
%%time
# Task 4

# Convert the Id columns to sets for efficient lookup (forcing computation)
question_ids = set(questions["Id"].compute())
tagged_question_ids = set(question_tags["Id"].compute())

# Check for dangling references
dangling_references = tagged_question_ids - question_ids

# Return 1 if there are dangling references, otherwise return 0
print(1 if len(dangling_references) > 0 else 0)



0
CPU times: user 11.5 s, sys: 3.96 s, total: 15.4 s
Wall time: 46.3 s


In [8]:
%%time
# Task 5

# Group by 'OwnerUserId' and compute required aggregations (without nunique)
owner_user_table = questions.groupby("OwnerUserId").agg(
    {"Score": "mean", "Id": "count", "AnswerCount": "sum"}
).rename(columns={"Score": "AverageScore", "Id": "NumQuestions", "AnswerCount": "NumAnswers"})

# Compute the result (convert to Pandas)
owner_user_table = owner_user_table.compute().reset_index()

# Get the top 5 users who asked the most questions
top_5_owners = owner_user_table.nlargest(5, "NumQuestions")

# Display the top 5 owner users
print("Top 5 Owner Users who asked the most questions:")
print(top_5_owners)


Top 5 Owner Users who asked the most questions:
        OwnerUserId  AverageScore  NumQuestions  NumAnswers
363219     875317.0      1.198206          2230      3499.0
9195        39677.0      6.607613          2128      5176.0
1972         4653.0      6.883095          1822      5696.0
9347        34537.0      5.213690          1680      4525.0
41352      179736.0      7.344987          1516      3982.0
CPU times: user 702 ms, sys: 196 ms, total: 898 ms
Wall time: 19 s


In [14]:
%%time
# Task 6

# Merge the two tables on 'Id' using Dask's merge
merged_table = question_tags.merge(questions, on="Id", how="inner")

# Group by 'Tag' and compute supported aggregations (without nunique)
tag_table = merged_table.groupby("Tag").agg(
    {"Score": "mean", "AnswerCount": "sum", "Id": "count"}
).rename(columns={"Score": "AverageScore", "AnswerCount": "NumAnswers", "Id": "NumQuestions"})

# Compute the result before calculating unique owner count
tag_table = tag_table.compute().reset_index()

# Compute unique owners per tag separately after converting to Pandas
num_owners = merged_table[["Tag", "OwnerUserId"]].drop_duplicates().groupby("Tag").size().compute()

# Convert num_owners to a DataFrame and merge with tag_table
num_owners = num_owners.to_frame(name="NumOwners").reset_index()
tag_table = tag_table.merge(num_owners, on="Tag", how="left")

# Display the first few rows of the new table
print(tag_table.head())

            Tag  AverageScore  NumAnswers  NumQuestions  NumOwners
0           php      0.375525   1855546.0       1365600     339456
1       plugins      0.903577     25199.0         27452      15256
2  architecture      2.018781     25229.0         16293       8955
3          hook      1.144506      4854.0          4851       3170
4          linq      1.824996    123939.0         69770      31406
CPU times: user 534 ms, sys: 58.4 ms, total: 592 ms
Wall time: 1min 58s


In [15]:
%%time
# Task 7: kill the third and fourth worker and repeat Task 5 with two workers 

# Group by 'OwnerUserId' and compute required aggregations (without nunique)
owner_user_table = questions.groupby("OwnerUserId").agg(
    {"Score": "mean", "Id": "count", "AnswerCount": "sum"}
).rename(columns={"Score": "AverageScore", "Id": "NumQuestions", "AnswerCount": "NumAnswers"})

# Compute the result (convert to Pandas)
owner_user_table = owner_user_table.compute().reset_index()

# Get the top 5 users who asked the most questions
top_5_owners = owner_user_table.nlargest(5, "NumQuestions")

# Display the top 5 owner users
print("Top 5 Owner Users who asked the most questions:")
print(top_5_owners)



Top 5 Owner Users who asked the most questions:
        OwnerUserId  AverageScore  NumQuestions  NumAnswers
363219     875317.0      1.198206          2230      3499.0
9195        39677.0      6.607613          2128      5176.0
1972         4653.0      6.883095          1822      5696.0
9347        34537.0      5.213690          1680      4525.0
41352      179736.0      7.344987          1516      3982.0
CPU times: user 236 ms, sys: 198 ms, total: 434 ms
Wall time: 31.7 s


Task 5 original time:
- CPU times: user 702 ms, sys: 196 ms, total: 898 ms
- Wall time: 19 s

Task 7 time: 
- CPU times: user 236 ms, sys: 198 ms, total: 434 ms
- Wall time: 31.7 s


Reasoning:


In [16]:
%%time
# Task 8: kill the third and fourth worker and repeat Task 6 with two workers 

# Merge the two tables on 'Id' using Dask's merge
merged_table = question_tags.merge(questions, on="Id", how="inner")

# Group by 'Tag' and compute supported aggregations (without nunique)
tag_table = merged_table.groupby("Tag").agg(
    {"Score": "mean", "AnswerCount": "sum", "Id": "count"}
).rename(columns={"Score": "AverageScore", "AnswerCount": "NumAnswers", "Id": "NumQuestions"})

# Compute the result before calculating unique owner count
tag_table = tag_table.compute().reset_index()

# Compute unique owners per tag separately after converting to Pandas
num_owners = merged_table[["Tag", "OwnerUserId"]].drop_duplicates().groupby("Tag").size().compute()

# Convert num_owners to a DataFrame and merge with tag_table
num_owners = num_owners.to_frame(name="NumOwners").reset_index()
tag_table = tag_table.merge(num_owners, on="Tag", how="left")

# Display the first few rows of the new table
print(tag_table.head())



            Tag  AverageScore  NumAnswers  NumQuestions  NumOwners
0           php      0.375525   1855546.0       1365600     339456
1       plugins      0.903577     25199.0         27452      15256
2  architecture      2.018781     25229.0         16293       8955
3          hook      1.144506      4854.0          4851       3170
4          linq      1.824996    123939.0         69770      31406
CPU times: user 451 ms, sys: 26.1 ms, total: 477 ms
Wall time: 3min 51s


Task 6 original time:
- CPU times: user 534 ms, sys: 58.4 ms, total: 592 ms
- Wall time: 1min 58s

Task 8 time:
- CPU times: user 451 ms, sys: 26.1 ms, total: 477 ms
- Wall time: 3min 51s


Reasoning: 